In [1]:
from selenium import webdriver
from pyshadow.main import Shadow
from selenium.webdriver.common.keys import Keys
import re
import time
import string
#Inputting english 5 letter words
dictionary = "american-english.txt"
#Assiging letters the letters
letters = set(string.ascii_letters)
noAttempt = 6
#Setting max length of word to 5 letters
lenWord = 5

In [2]:
#creating an array with all the words
wordlist = [word.strip() for word in open(dictionary, "r").readlines()]
#new array with only 5 letter words
tryWords = {
    word.lower()
    for word in wordlist    
    if len(word) == lenWord and set(word) < letters
    }
tryWords

{'noise',
 'udder',
 'buggy',
 'grain',
 'tooth',
 'motto',
 'gaily',
 'decal',
 'drool',
 'lathe',
 'scowl',
 'stint',
 'annex',
 'spiny',
 'agape',
 'using',
 'defer',
 'satyr',
 'recut',
 'credo',
 'ascot',
 'bluer',
 'range',
 'zonal',
 'fetus',
 'buyer',
 'lanky',
 'drawl',
 'nasal',
 'tonic',
 'known',
 'under',
 'hazel',
 'pizza',
 'merit',
 'leery',
 'annul',
 'aware',
 'flack',
 'clone',
 'goofy',
 'ionic',
 'ultra',
 'tower',
 'enjoy',
 'plead',
 'trawl',
 'toxic',
 'those',
 'twist',
 'welch',
 'aloft',
 'blank',
 'dense',
 'charm',
 'lefty',
 'ranch',
 'fixer',
 'rover',
 'block',
 'joust',
 'relax',
 'grant',
 'blitz',
 'debit',
 'detox',
 'thigh',
 'ovate',
 'canny',
 'gauze',
 'extol',
 'sooty',
 'jolly',
 'toddy',
 'modem',
 'cedar',
 'whack',
 'essay',
 'phone',
 'clamp',
 'trace',
 'field',
 'shore',
 'gassy',
 'askew',
 'latte',
 'scary',
 'sloth',
 'binge',
 'alibi',
 'globe',
 'linen',
 'tiger',
 'igloo',
 'leper',
 'track',
 'enema',
 'breed',
 'sewer',
 'ruder',


In [3]:
from collections import Counter
from itertools import chain

#counts the number of times a letter appears in the whole array
iterLetter = Counter(chain.from_iterable(tryWords))

iterLetter


Counter({'n': 575,
         'o': 754,
         'i': 671,
         's': 669,
         'e': 1233,
         'u': 467,
         'd': 393,
         'r': 899,
         'b': 281,
         'g': 311,
         'y': 425,
         'a': 979,
         't': 729,
         'h': 389,
         'm': 316,
         'l': 719,
         'c': 477,
         'w': 195,
         'x': 37,
         'p': 367,
         'f': 230,
         'z': 40,
         'k': 210,
         'j': 27,
         'v': 153,
         'q': 29})

In [4]:
#total number of letters in the list
total = len(tryWords)*lenWord
#divides the number of times a letter appears by the total number of letters
freqLetter = {character: value / total 
                    for character, value in iterLetter.items()}

freqLetter

{'n': 0.04967602591792657,
 'o': 0.0651403887688985,
 'i': 0.057969762419006476,
 's': 0.057796976241900645,
 'e': 0.10652267818574514,
 'u': 0.04034557235421166,
 'd': 0.033952483801295896,
 'r': 0.07766738660907127,
 'b': 0.02427645788336933,
 'g': 0.026868250539956805,
 'y': 0.0367170626349892,
 'a': 0.08457883369330453,
 't': 0.06298056155507559,
 'h': 0.033606911447084234,
 'm': 0.027300215982721383,
 'l': 0.06211663066954644,
 'c': 0.04120950323974082,
 'w': 0.016846652267818573,
 'x': 0.0031965442764578834,
 'p': 0.031706263498920084,
 'f': 0.019870410367170625,
 'z': 0.0034557235421166306,
 'k': 0.01814254859611231,
 'j': 0.002332613390928726,
 'v': 0.013218142548596112,
 'q': 0.002505399568034557}

In [5]:
def calculate_word_commonality(word):
    #intializes score to be 0
    score = 0
    for char in word:
        #add the freuqency of each character in the word to the score
        score += freqLetter[char]
    #returns the total score divided by 1
    return score / (lenWord - len(set(word)) + 1)

In [6]:
import operator

def sort_by_word_commonality(words):
    #sorts the words into descending order with highest score first
    sort_by = operator.itemgetter(1)
    return sorted(
        [(word, calculate_word_commonality(word)) for word in words],
        key = sort_by,
        reverse=True,
    )

def display_word_table(word_commonalities):
    #prints the words score
    for (word,freq) in word_commonalities:
        print(f"{word:<10} | {freq:<5.2}")


In [7]:
#rudementary method when scraping wordle didnt work
def input_word():
    while True:
        word = input("Whad'ya put in> ")
        if len(word) == lenWord and word.lower() in tryWords:
            break
    return word.lower()

def input_response():
    print("Put the following based on the response from Wordle:")
    print(" G for Green")
    print(" Y for Yellow")
    print(" ? for Gray")
    while True:
        response = input("Response from Wordle> ")
        if len(response) == lenWord and set(response) <= {"G", "Y", "?"}:
            break
        else:
            print(f"Error - invalid answer {response}")
    return response

In [8]:
def match_word_vector(word, word_vector):
    #they same length
    assert len(word) == len(word_vector)
    for letter, v_letter in zip(word, word_vector):
        #removing words with grey letters
        if letter not in v_letter:
            return False
    return True

def match(word_vector, possible_words):
    return [word for word in possible_words if match_word_vector(word, word_vector)]
    #returns possible words

In [9]:
def enter_word(browser, word):
    shadow = Shadow(browser)
    wordleBot = browser.find_element_by_tag_name('html')
    #finds the answer box
    wordleBot.click()
    time.sleep(1)
    wordleBot.send_keys(word)
    wordleBot.send_keys(Keys.ENTER)
    #autotypes and clicks enter
    time.sleep(1)
    finalEvaluation = list(word)
    #inputs the chosen word
    gameRow = shadow.find_element("game-row[letters="+word+"]")
    for char in set(word):
        gameTile = shadow.find_elements(gameRow,"game-tile[letter=" + char +"]")
        index = -1
        for elem in gameTile:
            if elem.get_attribute("evaluation") == 'present':
                finalEvaluation[word.index(char, index + 1, len(word))] = 'Y' 
            if elem.get_attribute("evaluation") == 'correct':
                finalEvaluation[word.index(char, index + 1, len(word))] = 'G'
            if elem.get_attribute("evaluation") == 'absent':
                finalEvaluation[word.index(char, index + 1, len(word))] = '?'
            index = word.index(char, index + 1, len(word))
    return finalEvaluation

In [13]:
def solve():
    browser = webdriver.Chrome(executable_path ='/Users/yui/go/yuiWordle/chromedriver')
    shadow = Shadow(browser)
    browser.get('http://www.powerlanguage.co.uk/wordle/')
    time.sleep(1)
    possible_words = tryWords.copy()
    word_vector = [set(string.ascii_lowercase) for _ in range(lenWord)]
    yLetters = set()
    for attempt in range(1, noAttempt + 1):
        #limits it to 6 tries
        print(f"Attempt {attempt} with {len(possible_words)} possible words")
        sortedWords = sort_by_word_commonality(possible_words)
        display_word_table(sortedWords[:15])
        i = 0
        word = sortedWords[0][0]
        #chosen word is the first in the sorted list hence one with the highest value
        while yLetters.issubset(set(word)) == False:
            i = i + 1
            word = sortedWords[i][0]
            #choses next best word with no greys
        response = enter_word(browser,word)
        for idx, letter in enumerate(response):
            if letter == "G":
                word_vector[idx] = {word[idx]}
            elif letter == "Y":
                try:
                    word_vector[idx].remove(word[idx])
                    yLetters.add(word[idx])
                except KeyError:
                    pass
            elif letter == "?":
                for vector in word_vector:
                    try:
                        vector.remove(word[idx])
                    except KeyError:
                        pass
        possible_words = match(word_vector, possible_words)
    

In [14]:
def solve():
    browser = webdriver.Chrome(executable_path ='/Users/yui/go/yuiWordle/chromedriver')
    shadow = Shadow(browser)
    browser.get('https://wordle.berknation.com/')
    time.sleep(10)
    possible_words = tryWords.copy()
    word_vector = [set(string.ascii_lowercase) for _ in range(lenWord)]
    yLetters = set()
    for attempt in range(1, noAttempt + 1):
        print(f"Attempt {attempt} with {len(possible_words)} possible words")
        sortedWords = sort_by_word_commonality(possible_words)
        display_word_table(sortedWords[:15])
        i = 0
        word = sortedWords[0][0]
        while yLetters.issubset(set(word)) == False:
            i = i + 1
            word = sortedWords[i][0]
        response = enter_word(browser,word)
        for idx, letter in enumerate(response):
            if letter == "G":
                word_vector[idx] = {word[idx]}
            elif letter == "Y":
                try:
                    word_vector[idx].remove(word[idx])
                    yLetters.add(word[idx])
                except KeyError:
                    pass
            elif letter == "?":
                for vector in word_vector:
                    try:
                        vector.remove(word[idx])
                    except KeyError:
                        pass
        possible_words = match(word_vector, possible_words)
    

In [17]:
solve()

<ipython-input-14-67c9c751dcff>:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(executable_path ='/Users/yui/go/yuiWordle/chromedriver')


Attempt 1 with 2315 possible words
alter      | 0.39 
alert      | 0.39 
later      | 0.39 
arose      | 0.39 
irate      | 0.39 
stare      | 0.39 
arise      | 0.38 
raise      | 0.38 
renal      | 0.38 
learn      | 0.38 
snare      | 0.38 
saner      | 0.38 
steal      | 0.37 
least      | 0.37 
slate      | 0.37 


<ipython-input-9-aedf243bdf53>:3: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  wordleBot = browser.find_element_by_tag_name('html')


QA--QAQA True
Attempt 2 with 196 possible words
sonic      | 0.27 
scion      | 0.27 
noisy      | 0.27 
disco      | 0.26 
bison      | 0.25 
sound      | 0.25 
synod      | 0.24 
bonus      | 0.24 
shiny      | 0.24 
spiny      | 0.23 
doing      | 0.23 
dingo      | 0.23 
minus      | 0.23 
using      | 0.23 
suing      | 0.23 
QA--QAQA True
Attempt 3 with 1 possible words
squib      | 0.18 
QA--QAQA True
Attempt 4 with 1 possible words
squib      | 0.18 
QA--QAQA True
Attempt 5 with 1 possible words
squib      | 0.18 
QA--QAQA True
Attempt 6 with 1 possible words
squib      | 0.18 
QA--QAQA True
